## Sharing data between processes
In multiprocessing, any newly created process will do following:
 * Run independently
 * Have their own memory space
 
 https://docs.python.org/3/library/multiprocessing.html#sharing-state-between-processes

<b> importing multiprocessing library

In [14]:
import multiprocessing

<B> Creating an empty list

In [16]:
result = []

In [17]:
def square(num_list):
    
    global result
    for num in num_list:
        result.append(num * num)
        
    print('Child process result:', result)

In [18]:
num_list = [1, 2, 3, 4]

In [19]:
p1 = multiprocessing.Process(target=square, args=(num_list, ))

p1.start()
p1.join()

Child process result: [1, 4, 9, 16]


<b> `output`:The child process makes a copy of the globaly declared list and append squares inside it that is why the updated list is only accessible inside the child process

In [20]:
print('Main process result:', result)

Main process result: []


<b> `output`: But main process is printing the initial empty list

## Method 1: Sharing Data Using Shared memory
 By using `shared memory` we can get rid of the issue of sharing memory among the processes. There is a memory region in our systems which is called `shared memory` and can be accessed by multiple proceeses, `Array` and `Value` objects are allocated from this shared memory. ( This array and value has no connection with python's array and value )

multiprocessing module provides Array and Value objects to share data between processes.
* Array: a ctypes array allocated from shared memory
* Value: a ctypes object allocated from shared memory

https://docs.python.org/3/library/multiprocessing.html#shared-ctypes-objects

###  The main agenda behind the following operation: 
* We are creating some variable in the main process, and changing them in the child process, after that we want to see that whether we can see the change in the variables if we call them from the main process

<b> We can not assign elements to a ctype array by using append method of python, we have to use enumerate that we take the help of index to assign value to the array

In [21]:
def square_list(numlist, result, square_sum):
    
    for idx, num in enumerate(numlist): 
        result[idx] = num * num 
        
    square_sum.value = sum(result)

<b> `multiprocessing.Array()` returns c type array as output <br> </b>
<b> `multiprocessing.Value()` retuerns c type object as output </b>

#### Initialize a shared Array
i denotes signed integer and 4 is the size of array to be allocated

In [22]:
result = multiprocessing.Array('i', 4)   

#### Initialize a shared Value
i is for the datatype of the object to be returned i.e signed integer

In [23]:
square_sum = multiprocessing.Value('i')  

In [24]:
num_list = [1, 2, 3, 4]

p = multiprocessing.Process(target = square_list, 
                            args = (num_list, result, square_sum))

In [25]:
p.start()
p.join()

In [26]:
list(result)

[1, 4, 9, 16]

In [27]:
square_sum.value

30

## Method 2: Sharing data using Server Process

Whenever a python program starts, a <b> server process </b> is also started. From there on, whenever a new process is needed, the parent process connects to the server and requests it to fork a new process. We can save the data in this server process which later can be shared among different child processes.
    

multiprocessing module provides a <b> Manager </b> class which controls a server process. Hence, this class provide way to share data using server process

https://github.com/nikhilkumarsingh/Parallel-Programming-in-Python/blob/master/06.%20Sharing%20data%20using%20Server%20Process/notebook.ipynb

In [28]:
def get_data(data_list):
    for data in data_list:
        print("Name: %s \nScore: %d\n" % (data[0], data[1]))

def append_data(new_data, data_list):
    data_list.append(new_data)
    print("New data appended!\n")

In [29]:
database = ([('Maura', 70), ('Alexis', 79), ('Pete',96)])

In [30]:
new_data = ('Leroy', 87)

In [31]:
p1 = multiprocessing.Process(target=append_data, 
                             args=(new_data, database))
p2 = multiprocessing.Process(target=get_data, args=(database, ))

In [32]:
p1.start()
p1.join()

New data appended!


In [33]:
p2.start()
p2.join()

Name: Maura 
Score: 70

Name: Alexis 
Score: 79

Name: Pete 
Score: 96


In [34]:
database

[('Maura', 70), ('Alexis', 79), ('Pete', 96)]

#### `output`: After execution of p2 process, the new data elements should be visible in the data list but it is not visible because while process start to work with variables they create their own copy of variable so the p1 process `new_data` is not the one which is printed by the p2 process. Without data sharing the processes can not coordinate with each other

### Now using the `Manager` class of multiprocessing module
Note that the database list here is of type manager.list()

In [35]:
with multiprocessing.Manager() as manager:
    
    database = (manager.list([('Maura', 70), ('Alexis', 79), ('Pete',96)]))
    new_data = ('Leroy', 87)

    p1 = multiprocessing.Process(target=append_data, 
                                 args=(new_data, database)) 
    p2 = multiprocessing.Process(target=get_data, args=(database,))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    print("Data available to the Manager: ", database)

New data appended!

Name: Maura 
Score: 70

Name: Alexis 
Score: 79

Name: Pete 
Score: 96

Name: Leroy 
Score: 87

Data available to the Manager:  [('Maura', 70), ('Alexis', 79), ('Pete', 96), ('Leroy', 87)]


<b> `output`: Process p2 asks server process for the updated list that is saved in the variable `data`, hence we can see the `new data` in it <b>